# Compte rendu de TP <br/>
Régression linéaire

## Rappel des rèlges de notation

Le TP de chaque étudiant sera évalué via la plateforme “Classgrade”. Pour cela, vous devez déposer
un unique fichier anonymisé (votre nom doit apparaître uniquement dans le nom du fichier lui-même)
sous format ipynb sur le site http://peergrade.enst.fr/.<br/>
Vous devez **charger votre fichier, avant le jeudi 18/09/2018, 23h59**. Entre le vendredi 19/09/2018 et
**le vendredi 26/09/2018, 23h59, vous devrez noter 2 copies** qui vous seront assignées anonymement, en
tenant compte du barème suivant pour chaque question du TP :
 * 0 (manquant/ non compris/ non fait/ insuffisant)
 * 1 (passable/partiellement satisfaisant)
 * 2 (bien)
 
Ensuite, il faudra également évaluer de la même manière les points suivants (qui correspondent à 3
questions supplémentaires) :
 * aspect global de présentation : qualité de rédaction, orthographe, présentation, graphes, titres,
etc...
 * aspect global du code : indentation, Style PEP8, lisibilité du code, commentaires adaptés
 * Point particulier : absence de bug sur votre machine
 
Des commentaires pourront être ajoutés question par question si vous en sentez le besoin ou l’utilité pour
aider la personne notée à s’améliorer, et de manière obligatoire si vous ne mettez pas 2/2 à une question.
**Enfin, veillez à rester polis et courtois dans vos retours.**

## Exercice:

### Sujet

(Analyse de la base de données “investment data”) La lecture d’un tutoriel
pandas pourra être utile : http://pandas.pydata.org/pandas-docs/stable/tutorials.html.
Nous travaillons sur la base de données Investment Data Set 1 qui peut être téléchargée depuis
https://bitbucket.org/portierf/shared_files/downloads/invest.txt.<br/> Avant de commencer, on
réalisera l’exercice 12 du polycopié : ”explicit formulas when p “ 1 for prediction intervals", se trouvant
dans le chapitre 3 : “Confidence intervals and hypothesis testing”. On pourra aussi lire la section 3.1.1 de
ce même chapitre du polycopié.

In [45]:
#Cellule d'import 
########################################################




#        ATTENTION CETTE ETAPE EST LONGUE





########################################################

# Instalation des packages potentiellement manquant et initialisation 
# S'il vous manque des packages décommentez les lignes nécessaires
import sys
envs = !conda env list
active_env = list(filter(lambda s: '*' in str(s), envs))[0]
env_name = str(active_env).split()[0]
print( env_name)
!conda install --yes -n  {env_name} plotly
#!conda install --yes -n  {env_name} numpy
#!conda install --yes -n  {env_name} pandas
#!conda install --yes -n  {env_name} scikit-learn
#!conda install --yes -n  {env_name} scipy
import plotly
print("fin d'installation ")
plotly.tools.set_credentials_file(username='telecommsbgd2', api_key='vycTMoJ16qDZEdUWiOiC')

# pandas
import pandas as pd

# maths et statistique
import numpy as np
from numpy.linalg import matrix_rank
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
import math

# graph
from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go

python36
Solving environment: done

# All requested packages already installed.

fin d'installation 


### Question 1

Importer la base de données “invest.txt” et l’afficher dans une forme lisible, e.g. une table contenant
les 5 premières observations.


In [46]:
investDF = pd.read_csv("https://bitbucket.org/portierf/shared_files/downloads/invest.txt", header=0,sep=" ")
print(investDF.head())

   year     gnp  invest     cpi  interest
0  1968   873.4   133.3   82.54      5.16
1  1969   944.0   149.3   86.79      5.87
2  1970   992.7   144.2   91.45      5.95
3  1971  1077.6   166.4   96.01      4.88
4  1972  1185.9   195.0  100.00      4.50


In [47]:
#Petit apperçu des données
investDF.describe(include='all')      

,year,gnp,invest,cpi,interest
count,15.000000,15.000000,15.000000,15.000000,15.000000
mean,1975.000000,1748.646667,276.006667,131.401333,7.452667
std,4.472136,738.145808,117.582691,40.286606,2.812245
min,1968.000000,873.400000,133.300000,82.540000,4.500000
25%,1971.500000,1131.750000,180.700000,98.005000,5.480000
50%,1975.000000,1549.200000,229.800000,125.790000,6.250000
75%,1978.500000,2290.850000,394.450000,156.920000,9.055000
max,1982.000000,3057.500000,471.500000,207.230000,13.420000


### Question 2

Réaliser le graph suivant : la variable “Gross National Product” (GNP, column “gnp”) est en abscisse
et la variable “Investment” (column “invest”) est en ordonnée. Transformer les 2 variables précédentes
en échelle logarithmique. Nous travaillerons désormais avec les 2 nouvelles variables.

In [48]:
# Changement d'echelle
investDF['invest'] = investDF.invest.apply(lambda x: math.log(x))
investDF['gnp'] = investDF.gnp.apply(lambda x: math.log(x))

In [49]:
# Définition de la courbe
data = [go.Scatter(x = investDF.gnp,
        y = investDF.invest,
        mode = 'markers')]

# Définition du design du graph
layout = go.Layout(title="Evolution of Investment depending on Gross National Produc",
                   yaxis=dict(title='Investment (log)'),
                   xaxis=dict(title='Gross National Produict (log)'))

# Attribution des informations pour la figure et affichage
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='Question-2')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~telecommsbgd2/0 or inside your plot.ly account where it is named 'Question-2'


In [50]:
# Si affichage ci-dessus non fonctionnel
#plt.plot(investDF.gnp,investDF.invest)
#plt.xlabel('Gross National Produc')
#plt.ylabel('Investment')
#plt.show()

NOTE : Lorsque l’on traite des données monétaires, on travaille souvent en échelle logarithmique (pour
prendre en compte les différences d’échelle).

### Question 3

Pour la régression de ”Investment” (variable à expliquer, output) sur ”GNP” (variable explicative,
covariable), estimer l’intercept et la pente, leurs écart-types, ainsi que le coefficient de détermination.
Les afficher dans une forme lisible. Dans la suite le vecteur contenant l’intercept et la pente est noté $\hat{\theta_n}$


In [51]:
n = investDF.shape[0]
Ymean = investDF.invest.mean()
Y = np.array(investDF.invest.values)
Xtildemean = investDF.gnp.mean()
Xtilde = np.array(investDF.gnp.values)
X = np.array([np.ones(n),Xtilde]).T


#Calcul de ma pente
beta1 = (Xtilde-Xtildemean).T @ (Y-Ymean)/((Xtilde-Xtildemean).T @ (Xtilde-Xtildemean))
print('beta1 estimé = '+ str(beta1)+" (correspond à la pente)")


#Calcul de l'intercept
beta0 = Ymean - beta1 * Xtildemean
print('beta0 estimé =' + str(beta0)+" (correspond à l'intercept)")
print()


# Définition de Teta^
Testime = np.array([beta0,beta1])


#Calcul du coefficient de determination
Yestime = X@Testime
Rsquare = ((Yestime-Ymean).T @ (Yestime-Ymean)) / ((Y-Ymean).T @ (Y-Ymean))
print("Le coéficient de determination vaut "+str(Rsquare))
print()


# Calcul de sigma estime
sigma_estime=np.sqrt(((Y-(X@Testime)).T @ (Y-(X@Testime)))/(n-matrix_rank(X)))


#Calculs des ecarts types
Vbeta0 = (sigma_estime**2) * ((1/n) + (Xtildemean * Xtildemean / ((Xtilde-Xtildemean).T @ (Xtilde-Xtildemean))))
Vbeta1 = (sigma_estime**2) * ((1 / ((Xtilde-Xtildemean).T @ (Xtilde-Xtildemean))))
print("L'écart type de beta0 vaut "+str(np.sqrt(Vbeta0)))
print("L'écart type de beta1 vaut "+str(np.sqrt(Vbeta1)))

beta1 estimé = 1.015281401598871 (correspond à la pente)
beta0 estimé =-1.9635913352301815 (correspond à l'intercept)

Le coéficient de determination vaut 0.9627572956057864

L'écart type de beta0 vaut 0.4095472716623086
L'écart type de beta1 vaut 0.055383060354852874


Les formules respectivement utilisées sont:  
beta1 $\Rightarrow \beta_1 = \frac{(X-\overline{X})^T (Y-\overline{Y})}{(X-\overline{X})^T (X-\overline{X})}   $  
beta0 $\Rightarrow \beta_0 = \overline{Y} - \beta1 * \overline{X}$  
Rsquare $\Rightarrow R^2 = \frac{(X-\overline{X})^T (Y-\overline{Y})}{(X-\overline{X})^T (X-\overline{X})}$   
Vbeta0 $\Rightarrow \mathbb{V}(\hat{\beta_0}) =  \hat{\sigma}^2 \left( \frac{1}{n} + \frac{\overline{X}^2}{(X-\overline{X})^T (X-\overline{X})} \right) $  
Vbeta1 $\Rightarrow \mathbb{V}(\hat{\beta_1}) =  \frac{\hat{\sigma}^2}{(X-\overline{X})^T (X-\overline{X})} $  

In [52]:
trace1 = go.Scatter(x = investDF.gnp,
            y = investDF.invest,
            mode = 'markers',
            name = "Original")
trace2 = go.Scatter(x = Xtilde,
            y = Yestime,
            name = "Estimé")
data = [trace1,trace2]

layout = go.Layout(title = "Estimation VS Realité",
                   yaxis = dict(title = 'Investment (log)'),
                   xaxis = dict(title = 'Gross National Produict (log)'))

fig = go.Figure(data = data, layout = layout)
py.iplot(fig, filename='Question-3')

### Question 4

La pente estimée précédemment est-elle statistiquement significative ? On fera un test de student
(t-test). Donner la valeur de la statistique de test ainsi que la p-valeur.

On pose $H_0$ l'hypothese que la pente estimée n'est pas statistiquement significative soit $\beta_1 = 0$ <br/>
Des lors l'hypothèse alternative peut etre défini comme $H_1 \Rightarrow \beta_1 \ne 0$

In [53]:
from scipy import stats as scs
Tj = beta1/(np.sqrt(Vbeta1))
p = 2 * (1 - scs.t.cdf(Tj, X.shape[0]-matrix_rank(X)))
print('La p-value vaut :'+str(p))

La p-value vaut :1.1363709973011282e-10


A partir de la $p-value$ on peut rejeter l'hypothese $H_0$

### Question 5
Pour GNP égal à 1000, estimer l’investissement prédit par le modèle. Pour GNP égal à 1000, donner l’intervalle de confiance pour la valeur prédite et l’intervalle de confiance pour la variable à expliquer “Investment”, au niveau 90%. On pourra se référer à la section 3.1.3 “Confidence intervals for the predicted values” du polycopié dans laquelle chaque intervalle est défini, $CI(x)$ et $PI(x)$, respectivement (avec les notations du polycopié, $x=(1, 1000)^T$ )

In [54]:
def predY(x,Teta):
    XX = np.array([1]+x)
    return XX.T @ Teta


Y1000 = predY([np.log(1000)],Testime)
print("Pour GNP égal à 1000 l’investissement prédit ( en réel ) vaut "+str(np.exp(Y1000)))
print("Pour GNP égal à 1000 l’investissement prédit ( en log ) vaut "+str(Y1000))

Pour GNP égal à 1000 l’investissement prédit ( en réel ) vaut 155.97942793105494
Pour GNP égal à 1000 l’investissement prédit ( en log ) vaut 5.049724126316802


En ayant $H_1 \Rightarrow \beta_1 \ne 0$, on suppose qu'on ne peut connaitre le signe de $\beta_1$ et nous nous trouvons dans un cas de bilatéralité

In [55]:

def getICIP(alpha,XX,X,Teta,sig):
    
    x = np.array([1]+XX)
    q = scs.t.ppf(1 - alpha/2, X.shape[0] - matrix_rank(X))
    
    Cinf = np.dot(np.transpose(x),(Teta)) - sig * q * np.sqrt(((x.T@np.linalg.inv(X.T@X))@x))
    Csup = np.dot(np.transpose(x),(Teta)) + sig * q * np.sqrt(((x.T@np.linalg.inv(X.T@X))@x))

    Pinf = np.dot(np.transpose(x),(Teta)) - sig * q * np.sqrt(1+((x.T@np.linalg.inv(X.T@X))@x))
    Psup = np.dot(np.transpose(x),(Teta)) + sig * q * np.sqrt(1+((x.T@np.linalg.inv(X.T@X))@x))
    
    return Cinf,Csup,Pinf,Psup

ICinf,ICsup,PIinf,PIsup=getICIP(.1,[np.log(1000)],X,Testime,sigma_estime)
print('IC = [' + str(ICinf)+',' + str(ICsup) + ']')
print('PI = [' + str(PIinf)+',' + str(PIsup) + ']')

    
    


IC = [4.988222845971108,5.111225406662496]
PI = [4.88282002343868,5.2166282291949235]


### Question 6
Sur un graphe avec échelle logarithmique, avec GNP en abscisse et investment en ordonnée, tracer les données, la droite de régression, ainsi que les intervalle CI et PI (pour toutes les valeurs de log(GNP) comprises entre le maximum et le minimum observé sur les données)

In [56]:
YPRED = []
ICSUP = []
ICINF = []
IPSUP = []
IPINF = []

XX=np.linspace(Xtilde.min(), Xtilde.max(), 500)
#calcul des IC, IP et des prédictions
for x in XX:
    YPRED.append(predY([x],Testime))
    ICinf_tmp,ICsup_tmp,PIinf_tmp,PIsup_tmp=getICIP(.1,[x],X,Testime,sigma_estime)
    ICSUP.append(ICsup_tmp)
    ICINF.append(ICinf_tmp)
    IPSUP.append(PIsup_tmp)
    IPINF.append(PIinf_tmp)
    

# attribution des courbes et nuage de points
trace0 = go.Scatter(
    x = XX[0::2],
    y = ICSUP[0::2],
    fill = 'tonexty',
    fillcolor = 'rgba(255, 215, 108,0.4)',
    line = dict(color = 'rgba(255,255,255,0)'),
    name = 'ICSUP',
    showlegend = True,
)
trace00 = go.Scatter(
    x = XX[0::2],
    y = ICINF[0::2],
    line = dict(color = 'rgba(255,255,255,0)'),
    name = 'ICINF',
    showlegend = False,
)
traceA = go.Scatter(
    x = XX[1::2],
    y = IPSUP[1::2],
    line = dict(color = 'rgba(255,255,255,0)'),
    fillcolor = 'rgba(0,176,246,0.2)',
    fill = 'tonexty',
    name = 'IPSUP',
    showlegend = True,
)
traceAA = go.Scatter(
    x = XX[1::2],
    y = IPINF[1::2],
    line = dict(color = 'rgba(255,255,255,0)'),
    name = 'IPINF',
    showlegend = False,
)
trace1 = go.Scatter(x = XX,
    y = YPRED,
    name = "Estimé")
trace2 = go.Scatter(x = Xtilde,
    y = Y,
    mode = "markers",
    name = "Original")
data = [traceAA,traceA,trace00,trace0,trace1,trace2]

layout = go.Layout(title = "Estimation VS Realité",
   yaxis = dict(title = 'Investment (log)'),
   xaxis = dict(title = 'Gross National Produict (log)'))

fig = go.Figure(data = data, layout = layout)
py.iplot(fig, filename='question-6')

### Querstion 7
En utilisant des classes/librairies existantes, donner l’intercept, la pente, le coefficient de dé- termination ainsi que l’investissement prédit par le modèle quand GNP vaut 100. La classe **LinearRegression() de sklearn.linear_model** est suggérée mais pas obligatoire. Vérifier que les valeurs calculées ici coïncident avec celles des questions précédentes.

In [57]:

regr = LinearRegression()
regr.fit(Xtilde.reshape(-1, 1), Y)


print("Intercept des librairies : "+str(regr.intercept_) +" Intercept calculé : "+str(beta0))
print("Pente des librairies : "+str(regr.coef_[0]) +" pente calculé : "+str(beta1))
print("Coefficient de détermination des librairies : "+str(regr.score(Xtilde.reshape(-1, 1), Y)) +" Coefficient de détermination calculé : "+str(Rsquare))
print("Y(1000) prédit des librairies : "+str(regr.predict([[np.log(1000)]])[0]) +" Y(1000) prédit calculé : "+str(Y1000))


Intercept des librairies : -1.9635913352301815 Intercept calculé : -1.9635913352301815
Pente des librairies : 1.015281401598871 pente calculé : 1.015281401598871
Coefficient de détermination des librairies : 0.9627572956057855 Coefficient de détermination calculé : 0.9627572956057864
Y(1000) prédit des librairies : 5.049724126316802 Y(1000) prédit calculé : 5.049724126316802


**On peut noter que nous obtenons les même valeurs précédement calculées**

### Question8
Sur un graph avec échelle logarithmique, avec GNP en abscisse et investment en ordonnée, tracer les données, la droite de régression, ainsi que l’investissement prédit par le modèle quand GNP vaut 1000 (on donnera à ce point une couleur différente).

In [58]:

YPREDlib = regr.predict(XX.reshape(-1, 1))

trace1 = go.Scatter(x = XX,
            y = YPREDlib,
            name = "Estimé")
trace2 = go.Scatter(x = Xtilde,
            y = Y,
            mode = 'markers',
            name = "Original")
trace3 = go.Scatter(x = [np.log(1000)],
            y = regr.predict([[np.log(1000)]]),
            mode = 'markers',
            name = 'Y1000',
            marker = dict(color = 'rgba(201, 29, 46,1)',
                size = 15),
            showlegend = False)
data = [trace1,trace2,trace3]


layout = go.Layout(title="Etude des valeurs prédites",
                   yaxis=dict(title = 'Investment (log)'),
                   xaxis=dict(title = 'Gross National Produict (log)'))

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Question-8')

### Question 9
Pour la régression de Investment sur GNP et interest, calculer la matrice de Gram. Est-elle de rang plein ?

In [59]:
n = investDF.shape[0]
X = np.array([np.ones(n),investDF.gnp,investDF.interest]).T
Xmean = np.array([1,investDF.gnp.mean(),investDF.interest.mean()]).T
mGram = np.dot(X.T,X)
print("Matrice de Gram")
print(mGram)
p
print("Le rang de la matrice de gram est "+ str(matrix_rank(mGram)))

Matrice de Gram
[[ 15.         110.75282116 111.79      ]
 [110.75282116 820.24847688 839.18854302]
 [111.79       839.18854302 943.8557    ]]
Le rang de la matrice de gram est 3


**La matrice de Gram est effectivement de rang plein.**

### Question 10

Pour la régression de Investment sur GNP et interest, estimer les 3 coefficients et leurs écart- types ainsi que le coefficient de détermination. En plus, faire un test de Student de significativité de chaque coefficient (donner la statistique de test et la p-valeur). Afficher les résultats dans une forme convenable. Discuter de la significativité des coefficients.

In [60]:

Ymean=investDF.invest.mean() 
Y=np.array(investDF.invest.values)


TetaEstime=np.linalg.inv(mGram) @ (X.T@Y)
print("Valeur de theta")
print(TetaEstime)
print()
            

#Calcul du coefficient de determination
Yestime=np.dot(X,TetaEstime)
Rsquare=(np.dot(np.transpose(Yestime-Ymean),Yestime-Ymean)/np.dot(np.transpose(Y-Ymean),Y-Ymean))
print("Le coéficient de determination vaut "+str(Rsquare))
print()

sigma_estime=np.sqrt(np.dot(np.transpose(Y-np.dot(X,TetaEstime)),Y-np.dot(X,TetaEstime))/(n-matrix_rank(X)))

#Calculs des ecarts types
covTeta=(sigma_estime**2)*np.linalg.inv(mGram)




print("L'écart type de beta0 vaut "+str(np.sqrt(covTeta[0][0])))
print("L'écart type de beta1 vaut "+str(np.sqrt(covTeta[1][1])))
print("L'écart type de beta2 vaut "+str(np.sqrt(covTeta[2][2])))

Valeur de theta
[-2.18045473  1.05124269 -0.00652888]

Le coéficient de determination vaut 0.9633108306727456

L'écart type de beta0 vaut 0.6623991977963686
L'écart type de beta1 vaut 0.10206137806192304
L'écart type de beta2 vaut 0.01534420506560419


On pose respectivement $H_{0a}$ l'hypothese que la feature gnp n'est pas statistiquement significative soit $\beta_0 = 0$, $H_{0b}$ l'hypothese que la feature gnp n'est pas statistiquement significative soit $\beta_1 = 0$ et $H_{0c}$ l'hypothese que la feature interest n'est pas statistiquement significative soit $\beta_2 = 0$<br/>
Des lors l'hypothèse alternative peut etre défini comme $$H_{1a} \Rightarrow \beta_0 \ne 0$$ $$H_{1b} \Rightarrow \beta_1 \ne 0$$ 
$$H_{1c} \Rightarrow \beta_2 \ne 0$$

In [61]:
from scipy import stats as scs
for i in range(3):
    Tj= TetaEstime[i]/(np.sqrt(covTeta[i][i]))
    Tj= TetaEstime[i]/(np.sqrt(covTeta[i][i]))
    Tj= TetaEstime[i]/(np.sqrt(covTeta[i][i]))
    print("beta"+str(i))
    print("T-statistique observé "+str(Tj))
    p =2*(1-scs.t.cdf(np.abs(Tj), X.shape[0] - matrix_rank(X)))
    print('La p-value vaut :'+str(p))
    print()

beta0
T-statistique observé -3.291753270786699
La p-value vaut :0.006438367050911831

beta1
T-statistique observé 10.30010285264071
La p-value vaut :2.5995168484449493e-07

beta2
T-statistique observé -0.4254947961276143
La p-value vaut :0.6780071418810365



Nos resultats sont cohérents avec les graphiques qui suivent. En effet on constate que la feature interest n'est pas significative pour déterminer l'investissement. Nous rejetons $H_{0b}$ et pouvons dire que $\beta_1 \ne 0$. Enfin on remarque que la p-value pour $\beta_0$ est assez importante cependant on ne rejete pas l'hypothese car la t-statistique reste dans la zone de rejet.

### Question 11
Pour les valeurs de GNP 1000 et interest 10, i.e., $x = (1, 1000, 10)^T$ , prédire log(investment) et donner les intervalles de confiance $CI(x)$ et $PI(x)$ au niveau 99.9%

In [62]:
ICinf,ICsup,PIinf,PIsup=getICIP(.001,[np.log(1000),10],X,TetaEstime,sigma_estime)
print('IC = [' + str(ICinf)+',' + str(ICsup) + ']')
print('PI = [' + str(PIinf)+',' + str(PIsup) + ']')

IC = [4.6401840366889235,5.3917834242781275]
PI = [4.473803343641108,5.558164117325943]


### Question 12
Sur un même graph à 3 dimensions avec les axes suivants : log(GNP), Interest, and log(Investment), tracer les données, le “plan” de régression et les surfaces correspondantes aux intervalles de confiance à 99.9% (ces surfaces seront tracées sur le domaine de définition des données).

In [63]:
# Initialisation et calcul pour affichage 3D
Xsample=np.linspace(investDF.gnp.min(), investDF.gnp.max(), 100)
Ysample=np.linspace(investDF.interest.min(), investDF.interest.max(), 100)

regr=[]
zICsup=[]
zICinf=[]
zIPsup=[]
zIPinf=[]

for i in Ysample:
    row=[]
    row1=[]
    row2=[]
    row3=[]
    row4=[]
    for j in Xsample:
        row.append(predY([j,i], TetaEstime))
        ICinf_tmp,ICsup_tmp,PIinf_tmp,PIsup_tmp=getICIP(.1,[j,i],X,TetaEstime,sigma_estime)
        row1.append(ICsup_tmp)
        row2.append(ICinf_tmp)
        row3.append(PIsup_tmp)
        row4.append(PIinf_tmp)
    regr.append(row)
    zICsup.append(row1)
    zICinf.append(row2)
    zIPsup.append(row3)
    zIPinf.append(row4)
regr=np.asarray(regr)

x_offset=np.min(Xsample)*np.ones(regr.shape)
y_offset=np.min(Ysample)*np.ones(regr.shape)
    
colorscale=[[0, 'gray'],[0.5,"orange"],  [1.0, 'red']]
colorscale2=[[0, 'blue'],  [1.0, 'blue']]
colorscale3=[[0, 'green'],  [1.0, 'green']]

# Creation des traces
trace1 = go.Scatter3d(
    x=investDF.gnp,
    y=investDF.interest,
    z=investDF.invest,
    mode='markers',
    name="Observation",
    showlegend=False,
    marker=dict(
        size=6,
        color=investDF.invest.sort_values(),
        opacity=0.8,
    )
)

trace2 = go.Surface(
        x=Xsample,
        y=Ysample,
        z=regr, 
        opacity=0.5,
        name="Plan de regression",
        colorscale=colorscale,
        showscale=False, 
        scene="scene"
        
    )

zICSUP = go.Surface(
        x=Xsample,
        y=Ysample,
        z=zICsup, 
        opacity=0.7,
        name="IC superieur",
        colorscale=colorscale3,
        showscale=False, 
        scene="scene"
        
    )
zICINF = go.Surface(
        x=Xsample,
        y=Ysample,
        z=zICinf, 
        opacity=0.7,
        name="IC inferieur",
        colorscale=colorscale3,
        showscale=False, 
        scene="scene"
        
    )

zIPSUP = go.Surface(
        x=Xsample,
        y=Ysample,
        z=zIPsup, 
        opacity=0.7,
        name="IP superieur",
        colorscale=colorscale2,
        showscale=False, 
        scene="scene"
        
    )
zIPINF = go.Surface(
        x=Xsample,
        y=Ysample,
        z=zIPinf, 
        opacity=0.7,
        name="IP inferieur",
        colorscale=colorscale2,
        showscale=False, 
        scene="scene"
        
    )



tracex = go.Surface(z=list(regr),
                x=list(x_offset),
                y=list(Ysample),
                name= 'Projection sur Interest',
                showscale=False,
                showlegend=False,
                colorscale=colorscale,
               )
tracey = go.Surface(z=list(regr),
                x=list(Xsample),
                y=list(y_offset),
                name='Projection sur GNP',
                showlegend=False,
                showscale=False,
                colorscale=colorscale,
                
               )


    
graphe1 = [trace1,trace2,tracex,tracey]
graphe2 = [trace1,trace2,zICINF,zICSUP,zIPSUP,zIPINF]

# Design du graphique
axis = dict(
showbackground=False, 
showgrid=True,    
zeroline=False,  
showline=False)
scene=dict(xaxis=dict(axis,title="Gross National Produict (log)"),
           yaxis=dict(axis,title="Interest"),
           zaxis=dict(title="Investment (log)"))

    

# Mise en place du graphique puis affichage
fig = tools.make_subplots(rows=1, cols=2,
                          specs=[[{'is_3d': True}, {'is_3d': True}]])
for var in graphe1:
    fig.append_trace(trace=var, row=1, col=1)
for var in graphe2:
    fig.append_trace(trace=var, row=1, col=2)


fig['layout']['scene1'].update(scene)
fig['layout']['scene2'].update(scene)
fig['layout']['title']="Evolution 3D de l'investissement selon les variables GNP et intéret"

py.iplot(fig, filename='Question-12')

This is the format of your plot grid:
[ (1,1) scene1 ]  [ (1,2) scene2 ]



Dans le graphique à gauche on remarque avec aisance que l'interet n'a que tres peu d'impact dans la prédiction de l'investissement contrairement au gross national product.  
On appréciera dans le graphique de droite la repartition des surfaces de $CI(x)$, $PI(x)$ et prédiction par rapport à nos valeurs originales.

### Question 13

En utilisant des classes/librairies existantes, donner les coefficients de régression, le coefficient de détermination ainsi que l’investissement prédit par le modèle quand GNP vaut 100 et interest 10. La classe LinearRegression() de sklearn.linear_model est suggérée mais pas obligatoire. Vérifier que les valeurs calculées ici coïncident avec celles des questions précédentes.


In [64]:

regr = LinearRegression()
Xtilde=np.array([investDF.gnp,investDF.interest]).T
regr.fit(Xtilde, Y)


print("beta0 des librairies : "+str(regr.intercept_) +" beta0 calculé : "+str(TetaEstime[0]))
print("beta1 des librairies : "+str(regr.coef_[0]) +" beta1 calculé : "+str(TetaEstime[1]))
print("beta2 des librairies : "+str(regr.coef_[1]) +" beta2 calculé : "+str(TetaEstime[2]))
print("Coefficient de détermination des librairies : "+str(regr.score(Xtilde, Y)) +" Coefficient de détermination calculé : "+str(Rsquare))
print("Y(1000,10) prédit des librairies : "+str(regr.predict([[np.log(1000),10]])[0]) +" Y(1000,10) prédit calculé : "+str(Y1000))


beta0 des librairies : -2.1804547259114004 beta0 calculé : -2.180454725912682
beta1 des librairies : 1.0512426913196595 beta1 calculé : 1.0512426913200557
beta2 des librairies : -0.006528879406068252 beta2 calculé : -0.006528879406129562
Coefficient de détermination des librairies : 0.9633108306726245 Coefficient de détermination calculé : 0.9633108306727456
Y(1000,10) prédit des librairies : 5.0159837304826835 Y(1000,10) prédit calculé : 5.049724126316802


**Avec les librairies nous pouvons vérifier la cohérence de nos calculs.**